# Materialized Gold Tables

Because the lakehouse combines on-demand compute resources with infinitely scalable cloud object storage to optimize cost and performance, the concept of a materialized view most closely maps to that of a gold table. Rather than caching the results to the view for quick access, results are stored in Delta Lake for efficient deserialization.

**NOTE**: Databricks SQL leverages <a href="https://docs.databricks.com/sql/admin/query-caching.html#query-caching" target="_blank">Delta caching and query caching</a>, so subsequent execution of queries will use cached results.

Gold tables refer to highly refined, generally aggregate views of the data persisted to Delta Lake.

These tables are intended to drive core business logic, dashboards, and applications.

The necessity of gold tables will evolve over time; as more analysts and data scientists use your Lakehouse, analyzing query history will reveal trends in how data is queried, when, and by whom. Collaborating across teams, data engineers and platform admins can define SLAs to make highly valuable data available to teams in a timely fashion, all while cutting down the potential costs and latency associated with larger ad hoc queries.

In this notebook, we'll create a gold table that stores summary statistics about each completed workout alongside binned demographic information. In this way, our application can quickly populate statistics about how other users performed on the same workouts.

<img src="https://files.training.databricks.com/images/ade/ADE_arch_bpm_summary.png" width="60%" />

## Learning Objectives
By the end of this lesson, students will be able to:
- Describe performance differences between views and tables
- Implement a streaming aggregate table

## Setup
Set up path and checkpoint variables (these will be used later).

In [0]:
%run ../Includes/Classroom-Setup-5.2

## Explore Workout BPM
Recall that our **`workout_bpm`** table has already matched all completed workouts to user bpm recordings.

Explore this data below.

In [0]:
%sql
SELECT * 
FROM workout_bpm
LIMIT 10

Here we calculate some summary statistics for our workouts.

In [0]:
%sql
SELECT user_id, workout_id, session_id, MIN(heartrate) min_bpm, MEAN(heartrate) avg_bpm, MAX(heartrate) max_bpm, COUNT(heartrate) num_recordings
FROM workout_bpm
GROUP BY user_id, workout_id, session_id

And now we can use our **`user_lookup`** table to match this back to our binned demographic information.

In [0]:
%sql
SELECT workout_id, session_id, a.user_id, age, gender, city, state, min_bpm, avg_bpm, max_bpm, num_recordings
FROM user_bins a
INNER JOIN 
  (SELECT user_id, workout_id, session_id, 
          min(heartrate) AS min_bpm, 
          mean(heartrate) AS avg_bpm,
          max(heartrate) AS max_bpm, 
          count(heartrate) AS num_recordings
   FROM workout_bpm
   GROUP BY user_id, workout_id, session_id) b
ON a.user_id = b.user_id

## Perform an Incremental Batch Table Update
Because our **`workout_bpm`** table was written as an append-only stream, we can update our aggregation using a streaming job as well.

In [0]:
(spark.readStream
      .table("workout_bpm")
      .createOrReplaceTempView("TEMP_workout_bpm"))

Using trigger-available-now logic with Delta Lake, we can ensure that we'll only calculate new results if records have changed in the upstream source tables.

In [0]:
user_bins_df = spark.sql("""
    SELECT workout_id, session_id, a.user_id, age, gender, city, state, min_bpm, avg_bpm, max_bpm, num_recordings
    FROM user_bins a
    INNER JOIN
      (SELECT user_id, workout_id, session_id, 
              min(heartrate) AS min_bpm, 
              mean(heartrate) AS avg_bpm, 
              max(heartrate) AS max_bpm, 
              count(heartrate) AS num_recordings
       FROM TEMP_workout_bpm
       GROUP BY user_id, workout_id, session_id) b
    ON a.user_id = b.user_id
    """)

(user_bins_df
     .writeStream
     .format("delta")
     .option("checkpointLocation", f"{DA.paths.checkpoints}/workout_bpm_summary")
     .option("path", f"{DA.paths.user_db}/workout_bpm_summary.delta")
     .outputMode("complete")
     .trigger(availableNow=True)
     .table("workout_bpm_summary")
     .awaitTermination())

## Query Results

Note that the primary benefit to scheduling updates to gold tables as opposed to defining views is the ability to control costs associated with materializing results.

While returning results from this table will use some compute to scan the **`workout_bpm_summary`** table, this design avoids having to scan and join files from multiple tables every time this data is referenced.

In [0]:
%sql
SELECT * FROM workout_bpm_summary

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()